<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

## <div style="text-align: left; background-color:#192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">TABLE OF CONTENTS</div>

* &nbsp; **[Used Libraries](#USED-LIBRARIES)**

* &nbsp; **[Metadata](#METADATA)**

* &nbsp; **[Load and Check Data](#LOAD-AND-CHECK-DATA)**

* &nbsp; **[Data Cleaning](#DATA-CLEANING)**

* &nbsp; **[Data Analysis](#EXPLORATORY-DATA-ANALYSIS)**

* &nbsp; **[K-Means Clustering](#EXPLORATORY-DATA-ANALYSIS)**

* &nbsp; **[Product Recommendation](#EXPLORATORY-DATA-ANALYSIS)**



<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

# <div style="text-align: left; background-color:#1192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">USED LIBRARIES</div>


*  **Numpy**      : For Mathematical Operations
*  **Pandas**     : For Data Processing and File Reading
*  **Seaborn**    : For Data Visualization
*  **MatplotLib** : For Data Visualization
*  **Sklearn**    : For Machine Learning Algorithms and Preprocessing
*  **Tkinter**    : For Building Graphical User Interfaces (GUI)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import tkinter as tk
from tkinter import ttk


<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

# <div style="text-align: left; background-color:#1192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">METADATA</div>

(From [Superstore Dataset](https://www.kaggle.com/datasets/vivek468/superstore-dataset-final) Data Card)

* Row ID ========> Unique ID for each row.
* Order ID ======> Unique Order ID for each Customer.
* Order Date ====> Order Date of the product.
* Ship Date =====> Shipping Date of the Product.
* Ship Mode =====> Shipping Mode specified by the Customer.
* Customer ID ===> Unique ID to identify each Customer.
* Customer Name => Name of the Customer.
* Segment =======> The segment where the Customer belongs.
* Country =======> Country of residence of the Customer.
* City ==========> City of residence of of the Customer.
* State =========> State of residence of the Customer.
* Postal Code ===> Postal Code of every Customer.
* Region ========> Region where the Customer belong.
* Product ID ====> Unique ID of the Product.
* Category ======> Category of the product ordered.
* Sub-Category ==> Sub-Category of the product ordered.
* Product Name ==> Name of the Product
* Sales =========> Sales of the Product.
* Quantity ======> Quantity of the Product.
* Discount ======> Discount provided.
* Profit ========> Profit/Loss incurred.

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

# <div style="text-align: left; background-color:#1192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">LOAD AND CHECK DATA</div>

In [ ]:
ss_data = pd.read_csv("Superstore-Data.csv")

In [ ]:
ss_data.head()

In [ ]:
ss_data.tail()

In [ ]:
ss_data.sample(5)

In [ ]:
ss_data.info()

## 

* **There are null values found in State column.**
* **There are 2 int64 columns ("Quantity" and "Postal Code")**
* **There are 3 float64 columns ("Sales", "Discount", "Profit")**
* **There are 15 object columns.**
* **We need to change data type for columns: Order Date, Ship Date, Postal Code.**

In [ ]:
ss_data[['Sales', 'Profit', 'Quantity', 'Discount']].describe()

In [ ]:
ss_data.describe(include='object')

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

# <div style="text-align: left; background-color:#1192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">DATA CLEANING</div>

## Check for inconsistencies

In [ ]:
ss_data.columns

In [ ]:
print(ss_data['Ship Mode'].unique())
print('------------------------------------------------')
print(ss_data['Segment'].unique())
print('------------------------------------------------')
print(ss_data['Category'].unique())
print('------------------------------------------------')
print(ss_data['Sub-Category'].unique())

In [ ]:
# to ensure that all column names are in lowercase and there is no space between letterws
ss_data.columns = ss_data.columns.str.lower().str.strip()

In [ ]:
ss_data.columns

In [ ]:
# make all data are lowercase and there is no space
for col in ss_data.select_dtypes(include=['object']).columns:
    ss_data[col] = ss_data[col].str.lower().str.strip()  

In [ ]:
print(ss_data[ss_data['sales'] < 0])

## Missing Data

In [ ]:
ss_data.isnull().sum()

In [ ]:
ss_data[ss_data['state'].isnull()]

In [ ]:
ss_data['state'] = ss_data['state'].fillna("vermont") # 05408 is Postal code for Vermont State 

In [ ]:
ss_data.isna().sum()

## Duplicate Data

In [ ]:
ss_data.duplicated().sum()

In [ ]:
ss_data.drop(columns=['row id'], inplace=True)

In [ ]:
ss_data.duplicated().sum()

In [ ]:
ss_data[ss_data.duplicated(keep=False)]

In [ ]:
ss_data = ss_data.drop_duplicates()

## Outliers

In [ ]:
sns.boxplot(x=ss_data['sales'])
plt.show()

In [ ]:
sns.boxplot(x=ss_data['profit'])
plt.show()

## Feature Engineering

In [ ]:
ss_data['order date'] = pd.to_datetime(ss_data['order date'], format='%m/%d/%Y')
ss_data['ship date'] = pd.to_datetime(ss_data['ship date'], format='%m/%d/%Y')
ss_data['postal code'] = ss_data['postal code'].astype(str)

In [ ]:
# Create a new feature: Days to Ship
ss_data['days to ship'] = (ss_data['ship date'] - ss_data['order date']).dt.days
ss_data.head()

## Save Cleaned Data

In [ ]:
#ss_data.to_csv('cleande_Superstore_data.csv', index=False)


<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

# <div style="text-align: left; background-color:#1192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">Exploratory Data Analysis (EDA)</div>

## Check Correlation

In [ ]:
# Heatmap to check correlation between numerical features
plt.figure(figsize=(10, 6))
sns.heatmap(ss_data.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()


## How many years dataset covers

In [ ]:
# Convert to datetime format
ss_data['order date'] = pd.to_datetime(ss_data['order date'])

# Find the range of years
start_year = ss_data['order date'].dt.year.min()
end_year = ss_data['order date'].dt.year.max()

# Get the number of unique years
num_years = ss_data['order date'].dt.year.nunique()

print(f"The data spans from {start_year} to {end_year}, covering {num_years} years.")

## What is the total number of customers?

In [ ]:
ss_data['customer id'].nunique()

## What is the total number of orders?

In [ ]:
ss_data['order id'].count()

## What is the total number of sold items?

In [ ]:
ss_data['quantity'].sum()

## What is the total number of Sales?

In [ ]:
sales_sum = ss_data['sales'].sum()
print(round(sales_sum, 2), '$')

## What is the total number of profit?

In [ ]:
profit_sum = ss_data['profit'].sum()
print(round(profit_sum, 2), '$')

## what is the profit ratio

In [ ]:

profit_ratio = (ss_data['profit'].sum() / ss_data['sales'].sum())*100
print(round(profit_ratio, 2),'%')

## What is Avg discount

In [ ]:
Avg_disc = ss_data['discount'].mean()*100
print(round(Avg_disc, 2), '%')

## Sales & profit by region

In [ ]:
sales_region = ss_data.groupby('region')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2)
print(sales_region)

# Plot bar chart
sales_region.plot(kind='bar', figsize=(10, 6), color=['steelblue', 'lightcoral'])

# Show plot
plt.show()

## Sales & profit by state

In [ ]:
sales_state = ss_data.groupby('state')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2).nlargest(10, 'sales')
print(sales_state)


# Visualize the impact of segments and shipping modes
sales_state.plot(kind='barh', stacked=True, figsize=(10, 6))

plt.show()

## Sales & profit by city

In [ ]:
sales_city = ss_data.groupby('city')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2).nlargest(20, 'sales')
print(sales_city)


# Plot bar chart
sales_city.plot(kind='bar', figsize=(10, 6), color=['steelblue', 'lightcoral'])

# Add a horizontal line at y=0 to highlight negative values
plt.axhline(0, color='red', linestyle='--', linewidth=1.5)

# Show plot
plt.show()

## Sales & profit by segment

In [ ]:
sales_segment = ss_data.groupby('segment')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2)
print(sales_segment)

sales_segment.plot(kind='bar', figsize=(10, 6), color=['steelblue', 'lightcoral'])

plt.show()

## Sales & profit by ship mode

In [ ]:
sales_ship_mode = ss_data.groupby('ship mode')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2)
print(sales_ship_mode)

sales_ship_mode.plot(kind='bar', figsize=(10, 6), color=['steelblue', 'lightcoral'])

## Sales & profit by category

In [ ]:
sales_category = ss_data.groupby('category')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2)
print(sales_category)

sales_category.plot(kind='bar', figsize=(10, 6), color=['steelblue', 'lightcoral'])


## Sales & profit by sub category

In [ ]:
sales_sub_category = ss_data.groupby('sub-category')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2)
print(sales_sub_category)

sales_sub_category.plot(kind='bar', figsize=(10, 6), color=['steelblue', 'lightcoral'])

# Add a horizontal line at y=0 to highlight negative values
plt.axhline(0, color='red', linestyle='--', linewidth=1.5)


## Sales & profit by product name

In [ ]:
sales_product_name = ss_data.groupby('product name')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2).nlargest(15, 'sales')
print(sales_product_name)

sales_product_name.plot(kind='barh', figsize=(10, 6), color=['steelblue', 'lightcoral'])

# Add a vertical line at x=0 to highlight negative values
plt.axvline(0, color='red', linestyle='--', linewidth=1.5)


## Sales & profit by customer name

In [ ]:
sales_customer_name = ss_data.groupby('customer name')[['sales', 'profit']].sum().sort_values(by='sales', ascending=False).round(2).nlargest(15, 'sales')
print(sales_customer_name)

sales_customer_name.plot(kind='barh', figsize=(10, 6), color=['steelblue', 'lightcoral'])

# Add a vertical line at x=0 to highlight negative values
plt.axvline(0, color='red', linestyle='--', linewidth=1.5)

## Sales & profit overtime (m\y)

In [ ]:
sales_profit_trend = ss_data.groupby(ss_data['order date'].dt.to_period('M'))[['sales', 'profit']].sum()
print(sales_profit_trend)
# Plot
sales_profit_trend.plot(kind='line', figsize=(12, 6))
# Add a horizontal line at zero to highlight negative profits
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.title("Sales & Profit Trend Over Time")
plt.ylabel("Amount")
plt.xlabel("Year-Month")
plt.show()


## Sales & profit over the year (seasonal trend)

In [ ]:
# Extract Month (1-12)
ss_data['month'] = ss_data['order date'].dt.month

# Aggregate total sales per month across all years
seasonal_sales_trend = ss_data.groupby('month')[['sales']].median()

# Plot the seasonal trend
plt.figure(figsize=(12, 6))
sns.lineplot(data=seasonal_sales_trend, x=seasonal_sales_trend.index, y='sales', marker='o')

# Formatting
plt.xticks(ticks=range(1, 13), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])
plt.title("Seasonal Sales Trend (Monthly Aggregation)")
plt.ylabel("Total Sales")
plt.xlabel("Month")
plt.grid(True)
plt.show()


## Impact of Shipping Delays on Sales 

In [ ]:
shipping_delay_impact = ss_data.groupby('days to ship')[['sales']].median()
print(shipping_delay_impact)

# Visualize the impact of shipping delays on sales and profit
shipping_delay_impact.plot(kind='line', figsize=(10, 6))
plt.title("Impact of Shipping Delays on Sales")
plt.ylabel("Amount")
plt.show()

## seasonal trend in Sales by Region

In [ ]:
# Extract month and year for seasonal analysis
ss_data['order_month'] =ss_data['order date'].dt.month
ss_data['order_year'] =ss_data['order date'].dt.year

# Group by region and month to see seasonal patterns
seasonal_sales =ss_data.groupby(['region', 'order_month'])[['sales']].median()

# Visualize seasonal sales by region
seasonal_sales.unstack(level=0).plot(kind='line', figsize=(12, 6), title=" seasonal trend in Sales by Region")
plt.ylabel("Sales")
plt.show()


## Impact of Discounts on Sales & Profitability

In [ ]:
discount_analysis = ss_data.groupby('discount')[['sales', 'profit']].sum()

plt.figure(figsize=(10, 6))

# Plot sales and profit against discount
discount_analysis.plot(kind='line', figsize=(10, 6))

plt.axhline(0, color='red', linestyle='--')
plt.title("Impact of Discounts on Sales & Profitability")
plt.ylabel("Amount")
plt.show()


## profit Over Years by category

In [ ]:
# Extract Year
ss_data['Year'] = ss_data['order date'].dt.year

# Grouping by Year and category
profit_overtime = ss_data.groupby(['Year', 'category'])[['profit']].sum().reset_index()

# Pivot to format for heatmap
profit_pivot = profit_overtime.pivot(index='Year', columns='category', values='profit')

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(profit_pivot, cmap="coolwarm", annot=True, fmt=".2f", linewidths=0.5)

# Formatting
plt.title("profit Over Years by category")
plt.xlabel("category")
plt.ylabel("Year")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


## profit Over Years by sub category

In [ ]:
# Extract Year
ss_data['Year'] = ss_data['order date'].dt.year

# Grouping by Year and sub-category
profit_overtime = ss_data.groupby(['Year', 'sub-category'])[['profit']].sum().reset_index()

# Pivot to format for heatmap
profit_pivot = profit_overtime.pivot(index='Year', columns='sub-category', values='profit')

# Plot heatmap
plt.figure(figsize=(18, 7))
sns.heatmap(profit_pivot, cmap="coolwarm", annot=True, fmt=".2f", linewidths=0.5)

# Formatting
plt.title("profit Over Years by sub-category")
plt.xlabel("sub-category")
plt.ylabel("Year")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist"></div>

# <div style="text-align: left; background-color:#1192AA; font-family: Trebuchet MS; color: #EEE8A9; padding: 13px; line-height:0.9;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 25px">Model Building</div>

In [ ]:
# Groping median Sales per Customer and customer id
customer_sales = ss_data.groupby('customer id', as_index=False)['sales'].median()

In [ ]:
# Scaling Sales Data
scaler = MinMaxScaler()
customer_sales['sales_scaled'] = scaler.fit_transform(customer_sales[['sales']])

In [ ]:
# Find Optimal Number of Clusters using Elbow Method
inertia = []
k_range = range(1, 11)  # Trying different K values from 1 to 10

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(customer_sales[['sales_scaled']])
    inertia.append(kmeans.inertia_)

# Plot Elbow Method Graph
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o', linestyle='-')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal K')
plt.show()

In [ ]:
# Apply K-Means Clustering with Optimal K (Choosed based on Elbow Method)
optimal_k = 4 
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
customer_sales['cluster'] = kmeans.fit_predict(customer_sales[['sales_scaled']])

In [ ]:
# Merge Cluster Labels Back into Original Dataset
ss_data = ss_data.merge(customer_sales[['customer id', 'cluster']], on='customer id', how='left')

In [ ]:
# Evaluating clustering
silhouette_avg = silhouette_score(customer_sales[['sales_scaled']], customer_sales['cluster'])
print(f'Silhouette Score: {silhouette_avg:.2f}')

In [ ]:
cluster_summary = customer_sales.groupby('cluster')['sales'].agg(['median', 'sum']).sort_values(by='sum')
print(cluster_summary)


## Product Recommendation

In [ ]:
# Function to recommend products for a user based on their cluster
def recommend_for_user(customer_id, top_n=10):
    if customer_id not in ss_data['customer id'].values:
        return ["No recommendations available."]

    # Find the cluster of the user
    user_cluster = ss_data.loc[ss_data['customer id'] == customer_id, 'cluster'].iloc[0]

    # Get all users in the same cluster
    cluster_users = ss_data.loc[ss_data['cluster'] == user_cluster, 'customer id'].unique()

    # Get product purchase history in this cluster
    cluster_purchases = ss_data.loc[ss_data['customer id'].isin(cluster_users), 'product name']

    if cluster_purchases.empty:
        return ["No purchase history available for this cluster."]

    # Find the most popular products in this cluster
    top_products = cluster_purchases.value_counts().index.tolist()

    # Find products the target user has already purchased
    user_purchases = ss_data.loc[ss_data['customer id'] == customer_id, 'product name'].unique()

    # Recommend new products (popular in cluster but NOT bought by user)
    recommendations = [product for product in top_products if product not in user_purchases]

    return recommendations[:top_n] if recommendations else ["No new recommendations available."]



In [ ]:
# Example usage
customer_id = "dv-13045"
recommended_products = recommend_for_user(customer_id)
print("Recommended products for customer", customer_id, ":", recommended_products)

<div style="
    background-color: #088299;
    color: #f7e6a2; 
    padding: 10px;
    text-align: center; 
    font-size: 20px; 
    font-weight: bold;
    border-radius: 5px; 
">
    Model saving
</div>


In [ ]:
import joblib
import sklearn

In [ ]:
joblib.dump(kmeans, 'kmeans_model.pkl')
joblib.dump(scaler, 'scaler.pkl')